<a id="intro_ID"></a>
# Intermediate: Portal Advanced Search for GI Program Light Curves

## Introduction
This notebook uses the MAST Portal's advanced search options to retrieve the light curves for a single guest investigator program.  The notbook will show how to do an advanced query on the MAST's database of holdings, determine which data files are associated with those observations and then download the files of interest.  

For more information about the TESS mission and collected data, visit the [MAST's TESS homepage](http://archive.stsci.edu/tess/). To read more details about TESS Data Products, look in the [TESS Science Product Description Document](https://archive.stsci.edu/missions/tess/doc/EXP-TESS-ARC-ICD-TM-0014.pdf). A list of Guest Investigator programs can be found at the [TESS GI List of Approved Programs](https://heasarc.gsfc.nasa.gov/docs/tess/approved-programs.html).

<a id="imports_ID"></a>
## Imports
Only the astroquery.mast Observations module is needed for this notebook.
For information on how to install astroquery see their [instructions](https://astroquery.readthedocs.io/en/latest/index.html). At the time of writing this requires the latest version on pip.

In [1]:
from astroquery.mast import Observations

We will use an advanced search to retrieve TESS timeseries data (lightcurve and target pixel files) for the Guest Investigator program G011183 from Sector 1 (PI: Stephen Kane). We will do this by first counting the number of observations returned and then requesting a table of those observations.

All the filter names are listed on the [MAST CAOM Field Description Page.](https://mast.stsci.edu/api/v0/_c_a_o_mfields.html). Or, can be found by hovering over the column names after doing a search in the MAST portal. 

The filters we will need to use are, **obs_collection** to specify that we want TESS data, **dataproduct_type** to specify that we want timeseries data, **sequence_number** to specify that we want sector 1 and **proopsal_id** to specify the GI program number.  Remember that more than one GI can propose for the same target so we need wild cards around the name of the program in our query.

In [15]:
obsCount = Observations.query_criteria_count(obs_collection="TESS",\
                                       dataproduct_type=["timeseries"],\
                                           sequence_number=1,\
                                          proposal_id="*G011183*")
print("Number of Observations: %i" % obsTable)

Number of Observations: 42


In [39]:
obsTable = Observations.query_criteria(obs_collection="TESS",\
                                       dataproduct_type=["timeseries"],\
                                           sequence_number=1,\
                                          proposal_id="*G011183*")
obsTable[0:5]['obsid','proposal_id','obs_id']


obsid,proposal_id,obs_id
str11,str31,str47
17000000913,G011112_G011183,tess2018206045859-s0001-0000000025155310-0120-s
17000011816,G011112_G011183,tess2018206045859-s0001-0000000231670397-0120-s
17000009457,G011112_G011183_G011132,tess2018206045859-s0001-0000000038846515-0120-s
17000011364,G011112_G011183_G011132,tess2018206045859-s0001-0000000149603524-0120-s
17000004054,G011112_G011183_G011132,tess2018206045859-s0001-0000000355703913-0120-s


### Retrieve the obserations
Next we use astroquery to retrieve the observations. More than one data file can be assocciated with a particular observation. So we will need to next ask for a list of products associated with the observations listed above.

We will only ask for the first two, but this could be run for all of the observations in the list.  We really just want to download the lightcurve files, so we will show how to do that. 

In [29]:
dataProducts = Observations.get_product_list(obsTable[0:2])
dataProducts.columns

<TableColumns names=('obsID','obs_collection','dataproduct_type','obs_id','description','type','dataURI','productType','productGroupDescription','productSubGroupDescription','productDocumentationURL','project','prvversion','proposal_id','productFilename','size','parent_obsid')>

In [38]:
dataProducts['obsID','productFilename','description']

obsID,productFilename,description
str11,str63,str27
17000000913,tess2018206190142-s0001-s0001-0000000025155310-00106_dvr.pdf,full data validation report
17000000913,tess2018206190142-s0001-s0001-0000000025155310-00106_dvr.xml,full data validation report
17000000913,tess2018206190142-s0001-s0001-0000000025155310-01-00106_dvs.pdf,TCE summary report
17000000913,tess2018206045859-s0001-0000000025155310-0120-s_lc.fits,Light curves
17000000913,tess2018206045859-s0001-0000000025155310-0120-s_tp.fits,Target pixel files
17000011816,tess2018206190142-s0001-s0001-0000000231670397-00106_dvr.pdf,full data validation report
17000011816,tess2018206190142-s0001-s0001-0000000231670397-00106_dvr.xml,full data validation report
17000011816,tess2018206190142-s0001-s0001-0000000231670397-01-00106_dvs.pdf,TCE summary report
17000011816,tess2018206045859-s0001-0000000231670397-0120-s_lc.fits,Light curves


In [34]:
want = dataProducts['description'] == "Light curves"
print(dataProducts[want])
manifest = Observations.download_products(dataProducts[want])

   obsID    obs_collection dataproduct_type ...   size  parent_obsid
----------- -------------- ---------------- ... ------- ------------
17000000913           TESS       timeseries ... 2039040  17000000913
17000011816           TESS       timeseries ... 2039040  17000011816


In [35]:
print(manifest)

                                                         Local Path                                                         ...
--------------------------------------------------------------------------------------------------------------------------- ...
./mastDownload/TESS/tess2018206045859-s0001-0000000025155310-0120-s/tess2018206045859-s0001-0000000025155310-0120-s_lc.fits ...
./mastDownload/TESS/tess2018206045859-s0001-0000000231670397-0120-s/tess2018206045859-s0001-0000000231670397-0120-s_lc.fits ...


## Summary
Here is a summary of the code needed to do the problem described above, without all the outputs getting in the way and bringing some of the query information to the top where it can easily be changed. 

In [40]:
GI_program="*G011183"
file_type="Light curves"
sector=1

obsTable = Observations.query_criteria(obs_collection="TESS",\
                                       dataproduct_type=["timeseries"],\
                                           sequence_number=sector,\
                                          proposal_id=GI_program)

dataProducts = Observations.get_product_list(obsTable[0:2])
want = dataProducts['description'] == file_type
manifest = Observations.download_products(dataProducts[want])

## Other Options
Some prefer to rely on the regularity of our directory structure and the TESS file names to directly download a file using curl, python requests etc. You can investigate this 